# Classifying Color Images - Dogs, Cats
### Author: Samuel Adamson
### Tensorflow, Google ML Datasets, Numpy, MatPlotLib
### Last Edited 01/01/2022

In [ ]:
# Imports
import tensorflow as tf
import tensorflow.keras.preprocessing.image as imageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import os

# Set logging for errors only
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Make directories
# !mkdir figs
# !mkdir model

Download Data from Google ML Dataset <br>
Unzip Data <br>
Store directories with different data types: <br>
&emsp; Training Data <br>
&emsp; Validation Data <br>
<br>
Evaluate data - Number of data points in each category

In [ ]:
# Data url for download
DATA_URL = 'https://storage/googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
# Download, store in directory, extract from .zip
dir = tf.keras.utils.get_file('cats_and_dogs_filtered.zip', origin=DATA_URL, extract=True)

# Store path to each type of data (training/validation)
data_dir = os.path.join(os.path.dirname(dir), 'cats_and_dogs_filtered')
train_dir = os.path.join(os.path.dirname(data_dir), 'train')
valid_dir = os.path.join(os.path.dirname(data_dir), 'validation')

# Store path to each type of image classification
train_cats_dir = os.path.join(os.path.dirname(train_dir), 'cats')
train_dogs_dir = os.path.join(os.path.dirname(train_dir), 'dogs')
valid_cats_dir = os.path.join(os.path.dirname(valid_dir), 'cats')
valid_dogs_dir = os.path.join(os.path.dirname(valid_dir), 'dogs')


# Number of validation images
num_val = len(os.listdir(valid_cats_dir)) + len(os.listdir(valid_dogs_dir))
print(f'Number of validation images: {num_val}')

# Number of training images
num_train = len(os.listdir(train_cats_dir)) + len(os.listdir(train_dogs_dir))
print(f'Number of training images: {num_train}')
print('---')

# Number of training / validation cats
num_train_cats = len(os.listdir(train_cats_dir))
num_valid_cats = len(os.listdir(valid_cats_dir))
tot_cats = num_train_cats + num_valid_cats
print(f'Number of training cat images: {num_train_cats}')
print(f'Number of validation cat images: {num_valid_cats}')
print(f'Total number of cat images: {tot_cats}')
print('---')

# Number of training / validation dogs
num_train_dogs = len(os.listdir(train_dogs_dir))
num_valid_dogs = len(os.listdir(valid_dogs_dir))
tot_dogs = num_train_dogs + num_valid_dogs
print(f'Number of training dog images: {num_train_dogs}')
print(f'Number of validation dog images: {num_valid_dogs}')
print(f'Total number of dog images: {tot_dogs}')

Preprocess Data / Prepare for training

In [ ]:
# Batch Size -- Number of images to process before adjusting model weights
BATCH_SIZE = 100
# Image Size -- height and width
IMG_SIZE = 150

# Image generators for training and validation sets
train_image_gen = imageDataGenerator(rescale=1.0/255.0)
valid_image_gen = imageDataGenerator(rescale=1.0/255.0)

# Flow images from directory into image generators
train_image_gen = train_image_gen.flow_from_directory(batch_size=BATCH_SIZE, directory=train_dir,
                                                      shuffle=True, target_size=(IMG_SIZE, IMG_SIZE), 
                                                      class_mode='binary')

valid_image_gen = train_image_gen.flow_from_directory(batch_size=BATCH_SIZE, directory=valid_dir,
                                                      shuffle=True, target_size=(IMG_SIZE, IMG_SIZE), 
                                                      class_mode='binary')                                    

Visualize Data

In [ ]:
# Get batch of images and labels from training set
sample_images, sample_labels = next(train_image_gen)

# Plot images and labels
# @PARAMS: Training data batch - images, labels
# @RETURN: None
def plotData(images, labels):
    # Set figure size
    plt.figure(figsize=(20,20))

    # Iterate through images
    for i, img in enumerate(images):
        # Resize image
        img = np.reshape(img, (28,28))
        # Plot Image
        plt.subplot(5,5,i+1)
        # Hide ticks
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        # Image and corresponding label
        plt.imshow(img, cmap=plt.cm.binary)
        plt.xlabel(labels[i])

    # Show pot
    plt.show()
    plt.savefig('./figs/example_data.png')

# Plot some test images
plotData(sample_images[:25], sample_labels[:25])

Create Model -- 4 Convolution Blocks 1 Densely Connected Layer

In [ ]:
# Create Model
model = tf.keras.models.Sequential([
    # Block 1
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # Block 2
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Block 3
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Block 4
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Dense layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

# Compile Model
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_lgits=True), 
              metrics=['accuracy']
)

# Display summary
model.summary()

Train Model